In [1]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()

# Project 4

### apps collection

In [110]:
batch = db.batch() #this is same as "Begin Transaction"

apps_df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = apps_df.values.tolist()

key_ben_df = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
key_ben_rows = key_ben_df.values.tolist()

reviews_df = pd.read_csv('shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
reviews_rows = reviews_df.values.tolist()

app_cat_df = pd.read_csv('shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
app_cat_rows = app_cat_df.values.tolist()

counter = 0

# Populating data
for app_row in apps_rows:
    
    record = {}
    record['id'] = app_row[0] #PK
    record['url'] = app_row[1]
    record['title'] = app_row[2]
    record['developer'] = app_row[3]
    record['developer_link'] = app_row[4]
    record['icon'] = app_row[5]
    record['rating'] = app_row[6]
    record['reviews_count'] = app_row[7]
    
    # Embed Key_Benefits
    key_ben_lst = []
    
    for kb_row in key_ben_rows:
        if app_row[0] == kb_row[0]:
            kb = {}
            kb['benefit title'] = kb_row[1]
            kb['description'] = kb_row[2]
            
            key_ben_lst.append(kb)
    
    if len(key_ben_lst) > 0:
        record['key_ben'] = key_ben_lst
        
    # Embed Reviews
    reviews_lst = []
    
    for reviews_row in reviews_rows:
        if app_row[0] == reviews_row[0]:
            rv = {}
            rv['author'] = reviews_row[1]
            rv['rating'] = reviews_row[2]
            rv['posted_on'] = reviews_row[3]
            
            reviews_lst.append(rv)
            
    if len(reviews_lst) > 0:
        record['reviews'] = reviews_lst
        
    
    # App_Category
    app_cat_lst = []
    
    for app_cat_row in app_cat_rows:
        if app_row[0] == app_cat_row[0]:
            app_cat_lst.append(app_cat_row[1]) #add category_id into apps collection
        
    if len(app_cat_lst) > 0:
        record['category_id'] = app_cat_lst
        
    
    apps_ref = db.collection('apps').document(app_row[0]) 
    batch.set(apps_ref, record) #Add documents
    counter += 1
    
    if counter == 100:
        batch.commit() #Commit
        counter = 0

### categories collection

In [19]:
batch = db.batch() #this is same as "Begin Transaction"

cat_df = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
cat_rows = cat_df.values.tolist()

# Populating data
for cat_row in cat_rows:
    
    record = {}
    record['id'] = cat_row[0] #PK
    record['cat_title'] = cat_row[1]
    
    # App_Category
    app_cat_lst2 = []
    
    for app_cat_row in app_cat_rows:
        if cat_row[0] == app_cat_row[1]:
            app_cat_lst2.append(app_cat_row[0]) #add app_id from app_category into apps collection
        
    if len(app_cat_lst2) > 0:
        record['app_id'] = app_cat_lst2
        
    cat_ref = db.collection('categories').document(cat_row[0]) 
    batch.set(cat_ref, record) #Add documents
    
    batch.commit() #Commit

### pricing_plans collection

In [108]:
batch = db.batch() #this is same as "Begin Transaction"

pp_df = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
pp_rows = pp_df.values.tolist()

ppf_df = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
ppf_rows = ppf_df.values.tolist()

# Populating data
for pp_row in pp_rows:
    
    record = {}
    record['id'] = pp_row[0] #PK
    record['app_id'] = pp_row[1]
    record['title'] = pp_row[2]
    record['price'] = pp_row[3]
    
    
    #Pricing Plan Features
    ppf_lst = []
    
    for ppf_row in ppf_rows:
        if pp_row[0] == ppf_row[0] and pp_row[1] == ppf_row[1]:
            ppf_lst.append(ppf_row[2]) 
    
    
    if len(ppf_lst) > 0:
        record['features'] = ppf_lst
        
    
    pp_ref = db.collection('pricing_plans').document(pp_row[0]) 
    batch.set(pp_ref, record) #Add documents
    
    batch.commit() #Commit
    
    

## Document counts on each collection

In [22]:
doc_apps = db.collection('apps').stream()

count = 0
for i in doc_apps:
    count += 1
    
print("There are {} documents in Apps collection".format(count))

There are 3500 documents in Apps collection


In [24]:
doc_cat = db.collection('categories').stream()

count=0
for i in doc_cat:
    count += 1
    
print("There are {} documents in Categories collection".format(count))

There are 12 documents in Categories collection


In [109]:
doc_pp = db.collection('pricing_plans').stream()

count = 0
for i in doc_pp:
    count += 1
    
print("There are {} documents in Pricing_Plans collection".format(count))

There are 6275 documents in Pricing_Plans collection


## Run queries

In [134]:
# 1
apps_ref = db.collection('apps')
query = apps_ref.order_by('reviews_count', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()

lst = ['id', 'title', 'developer', 'rating', 'reviews_count']

for result in results:
    output = {}
    dic =result.to_dict()
    for key in dic:
        if key in lst:
            output[key] = dic[key]
            
    print(f'{result.id} => {output}')


d9f142ee-b141-4dc4-9353-173db61d2eb0 => {'reviews_count': 23078, 'rating': 4.7, 'title': 'Privy ‑ Exit Pop Ups & Email', 'developer': 'Privy', 'id': 'd9f142ee-b141-4dc4-9353-173db61d2eb0'}
78ea0810-c008-4a4e-a82f-de0c790e3286 => {'reviews_count': 8737, 'rating': 4.9, 'developer': 'Hextom', 'title': 'Free Shipping Bar', 'id': '78ea0810-c008-4a4e-a82f-de0c790e3286'}
b88488b0-9912-44d3-b736-224c36f09d95 => {'reviews_count': 6905, 'rating': 4.8, 'title': 'Sales Pop ‑ Popup Notification', 'developer': 'CartKit', 'id': 'b88488b0-9912-44d3-b736-224c36f09d95'}
e528a60e-94f8-4e92-80e2-5bc6013b8283 => {'reviews_count': 5986, 'rating': 4.8, 'title': 'BEST Currency Converter', 'developer': 'Grizzly Apps', 'id': 'e528a60e-94f8-4e92-80e2-5bc6013b8283'}
be2640c4-01b5-4d52-9f68-cae8c0734d0d => {'id': 'be2640c4-01b5-4d52-9f68-cae8c0734d0d', 'reviews_count': 5596, 'rating': 4.8, 'title': 'Recart FB Messenger Marketing', 'developer': 'Recart'}
70bff9e0-4316-4cc6-84ce-92fcd1bc6925 => {'reviews_count': 525

In [107]:
#2
cat_ref = db.collection('categories').where('cat_title', '==', 'Trust and security')
cat_id = cat_ref.get()
cat_id = cat_id[0].to_dict()['id']


apps_ref = db.collection('apps')
query = apps_ref.where(
    'category_id', "array_contains", cat_id).order_by('rating', direction=firestore.Query.DESCENDING).limit(5)
results = query.stream()

lst = ['id', 'url', 'title', 'developer', 'rating']

for result in results:
    output = {}
    dic =result.to_dict()
    for key in dic:
        if key in lst:
            output[key] = dic[key]
            
    print(f'{result.id} => {output}')


fe2d6047-9654-494d-9eed-1f00e1468639 => {'rating': 5.0, 'developer': 'Ocean info', 'title': 'OC Check', 'id': 'fe2d6047-9654-494d-9eed-1f00e1468639', 'url': 'https://apps.shopify.com/age-checker-1'}
f5454a98-8bdb-499d-906a-bf8d9992b6e8 => {'id': 'f5454a98-8bdb-499d-906a-bf8d9992b6e8', 'url': 'https://apps.shopify.com/right-click-defender', 'rating': 5.0, 'developer': 'Adolab', 'title': 'Right Click Defender'}
ee95fc91-86c0-41b6-9d04-10422e0c42e8 => {'rating': 5.0, 'title': 'EU Cookie Consent Banner', 'developer': 'webserviceXXL', 'id': 'ee95fc91-86c0-41b6-9d04-10422e0c42e8', 'url': 'https://apps.shopify.com/eu-cookie-privacy-solution'}
e525d067-418e-49cb-b0c8-9081c85fe5bd => {'rating': 5.0, 'developer': 'Webkul Software Pvt Ltd', 'title': 'Social Commerce', 'id': 'e525d067-418e-49cb-b0c8-9081c85fe5bd', 'url': 'https://apps.shopify.com/social-e-commerce'}
e45d25ac-2939-4b88-9052-c6c033ff25d9 => {'developer': 'ZendApps', 'title': 'Geoip Product Blocker', 'id': 'e45d25ac-2939-4b88-9052-c6

In [222]:
#3
pp_ref = db.collection('pricing_plans').where('price', '>=', 12.99)
pp_ref = pp_ref.where('title', '==', 'Premium Plan').order_by('price', direction=firestore.Query.ASCENDING).limit(10)
results_pp = pp_ref.stream()

id_lst = []

for r in results_pp:
    
    d =r.to_dict()
    for key in d:
        if key == "app_id":
            id_lst.append(d[key])
            

for i in id_lst:
    apps_ref = db.collection('apps')
    query = apps_ref.where("id", "==", i)
    results = query.stream()           
    
    lst2 = ['id', 'title', 'developer', 'rating', 'reviews_count']
            
    for result in results:

        output = {}
        dic =result.to_dict()
        for key in dic:
            if key in lst2:
                output[key] = dic[key]

        print(f'{result.id} => {output}')        

a8171589-48cb-4a3a-83f1-5da7a20f5c03 => {'reviews_count': 95, 'rating': 4.9, 'title': 'Enorm Image Slider', 'developer': 'enormapps', 'id': 'a8171589-48cb-4a3a-83f1-5da7a20f5c03'}
18a74341-10cf-4a49-af8b-505ec047592d => {'rating': 5.0, 'title': 'Email Alchemy', 'developer': 'Oiizes', 'id': '18a74341-10cf-4a49-af8b-505ec047592d', 'reviews_count': 1}
ae85f088-0627-4368-8c2e-16be4cac506d => {'rating': 5.0, 'developer': 'Zooomy', 'title': 'Wishlist ‑ Wishify', 'id': 'ae85f088-0627-4368-8c2e-16be4cac506d', 'reviews_count': 161}
7b570802-fd42-49bf-a9f2-19d69838c049 => {'reviews_count': 527, 'rating': 4.7, 'title': 'Countdown Timer Bar', 'developer': 'Hextom', 'id': '7b570802-fd42-49bf-a9f2-19d69838c049'}
b1441cac-3052-4f43-aacc-9312d1d6ec6b => {'reviews_count': 0, 'rating': 0.0, 'title': 'Magic Ball ‑ Email Popup', 'developer': 'Application Nexus', 'id': 'b1441cac-3052-4f43-aacc-9312d1d6ec6b'}
3918da5e-152a-4a00-ad27-4fdfba27c9e5 => {'reviews_count': 103, 'rating': 4.8, 'developer': 'Mod Med